# Weather Informed Reservoir Operation

ResOpsUSData

- URL: https://zenodo.org/record/6612040
- Window: Daily records from 01-01-2008 to 12-31-2018 
- Steyaert, J.C., Condon, L.E., W.D. Turner, S. et al. ResOpsUS, a dataset of historical reservoir operations in the contiguous United States. Sci Data 9, 34 (2022). https://doi.org/10.1038/s41597-022-01134-7

California Data Exchange Center: California Department of Water Resources

- URL: https://cdec.water.ca.gov/dynamicapp/wsSensorData 
- Window: Daily and hourly records from 01-01-2008 to 12-31-2018 

In [73]:
import warnings
warnings.simplefilter("ignore")

from datetime import datetime, date
import pandas as pd
import numpy as np

import seaborn as sns
sns.set_theme(style="white")
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from math import sqrt

In [142]:
PROJECT_PATH = 'Projects/ML_Earth_Projects/Reservoir_Project'

### Loading ResOps data 

In [3]:
# from time_series_single_variable_table

daily_inflow = pd.read_csv(f'{PROJECT_PATH}/ResOpsUS/time_series_single_variable_table/DAILY_AV_INFLOW_CUMECS.csv')

In [4]:
daily_inflow.head()

,date,100,1000,1001,1002,1003,1005,1006,1007,101,...,981,982,983,987,989,99,991,993,995,998
0,1980-01-01,NaN,NaN,1.416,NaN,1.699,NaN,9.911,1.416,NaN,...,NaN,0.142,NaN,NaN,22.653,NaN,NaN,NaN,NaN,NaN
1,1980-01-02,NaN,NaN,1.416,NaN,1.699,NaN,8.495,1.416,NaN,...,NaN,0.142,NaN,NaN,22.653,NaN,NaN,NaN,NaN,NaN
2,1980-01-03,NaN,NaN,1.416,NaN,1.416,NaN,8.495,1.416,NaN,...,NaN,0.142,NaN,NaN,24.069,NaN,NaN,NaN,NaN,NaN
3,1980-01-04,NaN,NaN,1.416,NaN,0.850,NaN,7.079,0.850,NaN,...,NaN,0.142,NaN,NaN,24.069,NaN,NaN,NaN,NaN,NaN
4,1980-01-05,NaN,NaN,1.416,NaN,0.850,NaN,5.663,0.850,NaN,...,NaN,0.142,NaN,NaN,19.822,NaN,NaN,NaN,NaN,NaN


In [5]:
daily_inflow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Columns: 679 entries, date to 998
dtypes: float64(678), object(1)
memory usage: 77.7+ MB


In [6]:
# from attributes

time_series_inventory = pd.read_csv(f'{PROJECT_PATH}/ResOpsUS/attributes/time_series_inventory.csv')
time_series_inventory_variables = pd.read_csv(f'{PROJECT_PATH}/ResOpsUS/attributes/time_series_inventory_variables.csv')
time_series_variables = pd.read_csv(f'{PROJECT_PATH}/ResOpsUS/attributes/time_series_variables.csv')

reservoir_attributes = pd.read_csv(f'{PROJECT_PATH}/ResOpsUS/attributes/reservoir_attributes.csv')
reservoir_attributes_variables = pd.read_csv(f'{PROJECT_PATH}/ResOpsUS/attributes/reservoir_attributes_variables.csv')

agency_attributes = pd.read_csv(f'{PROJECT_PATH}/ResOpsUS/attributes/agency_attributes.csv')
agency_attributes_variables = pd.read_csv(f'{PROJECT_PATH}/ResOpsUS/attributes/agency_attributes_variables.csv')

In [7]:
time_series_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   DAM_ID             678 non-null    int64 
 1   STORAGE            678 non-null    int64 
 2   STORAGE_START      653 non-null    object
 3   STORAGE_END        653 non-null    object
 4   DATA_SOURCE        653 non-null    object
 5   INFLOW             678 non-null    int64 
 6   INFLOW_START       316 non-null    object
 7   INFLOW_END         316 non-null    object
 8   DATA_SOURCE.1      332 non-null    object
 9   OUTFLOW            678 non-null    int64 
 10  OUTFLOW_START      518 non-null    object
 11  OUTFLOW_END        518 non-null    object
 12  DATA_SOURCE.2      555 non-null    object
 13  ELEVATION          678 non-null    int64 
 14  ELEVATION_START    506 non-null    object
 15  ELEVATION_END      506 non-null    object
 16  DATA_SOURCE.3      553 non-null    object
 1

In [8]:
time_series_inventory.head()

,DAM_ID,STORAGE,STORAGE_START,STORAGE_END,DATA_SOURCE,INFLOW,INFLOW_START,INFLOW_END,DATA_SOURCE.1,OUTFLOW,...,OUTFLOW_END,DATA_SOURCE.2,ELEVATION,ELEVATION_START,ELEVATION_END,DATA_SOURCE.3,EVAPORATION,EVAPORATION_START,EVAPORATION_END,DATA_SOURCE.4
0,41,1,10/1/70,8/31/20,SCL,1,10/1/70,8/31/20,SCL,1,...,8/31/20,SCL,1,10/1/70,8/31/20,SCL,0,NaN,NaN,NaN
1,42,1,10/1/70,8/31/20,SCL,1,10/1/70,8/31/20,SCL,1,...,8/31/20,SCL,1,10/1/70,8/31/20,SCL,0,NaN,NaN,NaN
2,54,1,10/1/15,12/31/20,BOR_PN,0,NaN,NaN,NaN,1,...,12/31/20,BOR_PN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
3,55,1,1/1/65,12/31/20,BOR_PN,1,1/1/65,12/31/20,BOR_PN,1,...,12/31/20,BOR_PN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
4,56,1,1/1/90,12/31/20,ACE_SEA,0,NaN,NaN,NaN,1,...,12/31/20,usgs_12105900,1,1/1/90,12/31/20,ACE_SEA,0,NaN,NaN,NaN


In [9]:
time_series_inventory_variables

,Variable,Meaning
0,DAM_ID,ID for the dam which can be linked to time ser...
1,STORAGE,Binary to depict if data exists for this varia...
2,STORAGE_START,Earliest date that there is storage data for t...
3,STORAGE_END,Last date that there is storage data for this ...
4,DATA_SOURCE,The data source of this variable. Note that da...
5,INFLOW,Binary to depict if data exists for this varia...
6,INFLOW_START,Earliest date that there is inflow data for th...
7,INFLOW_END,Last date that there is inflow data for this d...
8,OUTFLOW,Earliest date that there is outflow data for t...
9,OUTFLOW_END,Last date that there is outflow data for this ...


In [10]:
time_series_variables

,Variable,Unit,Unit Text,Time Period,Parameter
0,IN,ms,cubic meters per second,Daily,Reservoir Inflow
1,STOR,MCM,Million Cubic Meters,Daily,Reservoir Storage
2,ELE,m,meters,Daily,Reservoir Elevation
3,OUT,ms,cubic meters per second,Daily,Reservoir Outflow
4,EVAP,MCM,million cubic meters,Daily,Reservoir Evaporation


In [11]:
reservoir_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DAM_ID                 678 non-null    int64  
 1   DAM_NAME               678 non-null    object 
 2   STATE                  678 non-null    object 
 3   AGENCY_CODE            678 non-null    object 
 4   LONG                   678 non-null    float64
 5   LAT                    678 non-null    float64
 6   TIME_SERIES_START      678 non-null    object 
 7   TIME_SERIES_END        677 non-null    object 
 8   INCONSISTENCIES_NOTED  3 non-null      object 
dtypes: float64(2), int64(1), object(6)
memory usage: 47.8+ KB


In [12]:
reservoir_attributes.head()

,DAM_ID,DAM_NAME,STATE,AGENCY_CODE,LONG,LAT,TIME_SERIES_START,TIME_SERIES_END,INCONSISTENCIES_NOTED
0,41,Ross,Washington,SCL,-121.067305,48.732466,10/1/70,8/31/20,NaN
1,42,Diablo,Washington,SCL,-121.130320,48.714583,10/1/70,8/31/20,NaN
2,54,Wynoochee,Washington,USGS,-123.604561,47.386357,1/1/80,12/31/20,NaN
3,55,Keechelus,Washington,BOR_PN,-121.340171,47.323654,1/1/65,12/31/20,NaN
4,56,Howard A. Hanson Dam,Washington,ACE_SEA,-121.783969,47.277406,1/1/80,12/31/20,NaN


In [13]:
reservoir_attributes_variables.head()

,Variable,Meaning
0,DAM_ID,Unique DAM_ID for each dam that links reservoi...
1,DAM_NAME,Name of the dam that corresponds to the GRAND_...
2,STATE,State where the dam is located.
3,LONG,The longitude of the dam's location.
4,LAT,The latitude of the dam's location.


In [14]:
agency_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Agency_Code            47 non-null     object
 1   Agency_Name            47 non-null     object
 2   Data_Disclaimers       8 non-null      object
 3   Inflow/Outflow         44 non-null     object
 4   Additional_Data_Notes  16 non-null     object
 5   Access_ Date           46 non-null     object
 6   Access_Type            46 non-null     object
dtypes: object(7)
memory usage: 2.7+ KB


In [15]:
agency_attributes.head()

,Agency_Code,Agency_Name,Data_Disclaimers,Inflow/Outflow,Additional_Data_Notes,Access_ Date,Access_Type
0,BOR,Bureau of Reclamation,NaN,Real time,NaN,Spring 2020 - Fall 2020,website download: https://www.usbr.gov/rsvrWat...
1,ACE_LOU,Army Corps of Engineers- Louisville District,Disclaimer: Data may not be 100% accurate,Aggregated spatially and temporally,Given inflow and outflow from multiple sources...,Spring 2020,personal communication
2,ACE_SAC,Army Corps of Engineers-Sacramento District,NaN,NaN,NaN,Spring 2021,website download:https://nicholasinstitute.duk...
3,ACE_OMA,Army Corps of Engineers-Omaha District,NaN,NaN,NaN,Summer 2020,personal communication
4,ACE_STP,Army Corps of Engineers-St. Paul District,All data contained herein is preliminary in na...,NaN,NaN,NaN,NaN


In [16]:
agency_attributes_variables

,Variable,Meaning
0,Agency_Code,Unique code for each agency which can be trace...
1,Agency_Name,"Agency name and, where applicable, district na..."
2,Data_Disclaimers,Disclaimers on data use provided directly by t...
3,Inflow/Outflow,This columns tells if inflow and/or outflow we...
4,Additional_Data_Notes,Any additional notes for the processing or the...
5,Acces_ Date,Date when the data was last accessed.
6,Access_Type,Personal communication denotes that the data c...


### Selecting inflow data

In [17]:
# start with California dataset

ca_reservoirs = reservoir_attributes.loc[reservoir_attributes["STATE"] == "California", ["DAM_ID", "DAM_NAME", "AGENCY_CODE", "LAT", "TIME_SERIES_START", "TIME_SERIES_END"]]

In [18]:
ca_reservoirs[ca_reservoirs["DAM_NAME"] == "Folsom"]

,DAM_ID,DAM_NAME,AGENCY_CODE,LAT,TIME_SERIES_START,TIME_SERIES_END
76,182,Folsom,BOR,38.710426,3/1/55,12/18/18


In [19]:
folsom_inventory = time_series_inventory[time_series_inventory["DAM_ID"] == 182]

folsom_inventory_outflow_start = folsom_inventory.loc[:, ['DAM_ID', 'INFLOW_START', 'INFLOW_END']]
folsom_inventory_outflow_start

,DAM_ID,INFLOW_START,INFLOW_END
76,182,3/1/55,12/31/20


In [20]:
"""
INFLOW_START: 3/1/55
INFLOW_END: 12/31/20
"""

folsom_daily_inflow = daily_inflow.loc[:, ['date', '182']]
folsom_daily_inflow['date'] = folsom_daily_inflow['date'].map(lambda d: pd.Timestamp(d))

In [21]:
# index for 2008-1-1 is 10227 
# 182 is the dam ID 

folsom_daily_inflow[folsom_daily_inflow["date"] == datetime(2008, 1, 1)]

,date,182
10227,2008-01-01,27.666


In [22]:
# index for 2018-12-31 is 14262

folsom_daily_inflow[folsom_daily_inflow["date"] == datetime(2018, 12, 31)]

,date,182
14262,2018-12-31,33.895


In [23]:
# 1/1/2008 - 12/31/18 is the inflow range for the reservoir to use 
# Using 20 years worth of data 

INFLOW_FROM_2008 = folsom_daily_inflow.iloc[10227:14263]
INFLOW_FROM_2008.reset_index(drop=True, inplace=True)
INFLOW_FROM_2008.rename(columns={"date": "Date", "182": "Inflow"}, inplace=True)
INFLOW_FROM_2008.head()

,Date,Inflow
0,2008-01-01,27.666
1,2008-01-02,30.044
2,2008-01-03,48.478
3,2008-01-04,90.246
4,2008-01-05,200.200


In [24]:
INFLOW_FROM_2008.tail()

,Date,Inflow
4031,2018-12-27,58.559
4032,2018-12-28,46.808
4033,2018-12-29,45.250
4034,2018-12-30,42.985
4035,2018-12-31,33.895


### Loading and selecting CDEC Water data 

In [117]:
# Prepare dataframes for merge

def prepare_station_df(station_data, hourly=False):
    # Convert observed dates to shared date format
    if str(station_data['OBS DATE'].dtype) == "int64":
      station_data['OBS DATE'] = pd.to_datetime(station_data['OBS DATE'], format='%Y%m%d')
    else: 
      station_data['OBS DATE'] = pd.to_datetime(station_data['OBS DATE']).dt.date
    
    # Set date as index
    if (hourly):
      station_data = station_data.groupby("OBS DATE").mean()
    else: 
      station_data.set_index('OBS DATE', inplace=True)

    # Create station df with observed value
    df = station_data[['VALUE']].copy()

    return df

#### Near American River, North Fork

In [118]:
### Auburn Dam Ridge (ADR) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
ADR_PRECIP_ACC_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/ADR/ADR_2.xlsx')
ADR_PRECIP_ACC_DF = prepare_station_df(ADR_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
# Incremental precipitation = difference between current and preceding accumulated precipitation
ADR_PRECIP_INCR_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/ADR/ADR_45.xlsx')
ADR_PRECIP_INCR_DF = prepare_station_df(ADR_PRECIP_INCR_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
ADR_TEMP_AVG_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/ADR/ADR_30.xlsx')
ADR_TEMP_AVG_DF = prepare_station_df(ADR_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
ADR_TEMP_MAX_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/ADR/ADR_31.xlsx')
ADR_TEMP_MAX_DF = prepare_station_df(ADR_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
ADR_TEMP_MIN_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/ADR/ADR_32.xlsx')
ADR_TEMP_MIN_DF = prepare_station_df(ADR_TEMP_MIN_DATA)


In [112]:
ADR_TEMP_MIN_DF.head()

,ADR_TEMP_MIN
OBS DATE,
2008-01-01,34.0
2008-01-02,39.0
2008-01-03,38.0
2008-01-04,42.0
2008-01-05,37.0


In [119]:
# Reset value name for df merge

ADR_PRECIP_ACC_DF.rename(columns={"VALUE": "ADR_PRECIP_ACC"}, inplace=True)
ADR_PRECIP_INCR_DF.rename(columns={"VALUE": "ADR_PRECIP_INCR"}, inplace=True)
ADR_TEMP_AVG_DF.rename(columns={"VALUE": "ADR_TEMP_AVG"}, inplace=True)
ADR_TEMP_MAX_DF.rename(columns={"VALUE": "ADR_TEMP_MAX"}, inplace=True)
ADR_TEMP_MIN_DF.rename(columns={"VALUE": "ADR_TEMP_MIN"}, inplace=True)

In [79]:
ADR_TEMP_MIN_DF.head()

,ADR_TEMP_MIN
OBS DATE,
2008-01-01,34.0
2008-01-02,39.0
2008-01-03,38.0
2008-01-04,42.0
2008-01-05,37.0


In [120]:
### Huysink (HYS) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
HYS_PRECIP_ACC_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/HYS/HYS_2.xlsx')
HYS_PRECIP_ACC_DF = prepare_station_df(HYS_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
HYS_PRECIP_INCR_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/HYS/HYS_45.xlsx')
HYS_PRECIP_INCR_DF = prepare_station_df(HYS_PRECIP_INCR_DATA)

# SNOW DEPTH, INCHES (daily)
HYS_SNOW_DEPTH_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/HYS/HYS_18.xlsx')
HYS_SNOW_DEPTH_DF = prepare_station_df(HYS_SNOW_DEPTH_DATA)

# SNOW, WATER CONTENT, INCHES (daily)
HYS_SNOW_WATER_CONTENT_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/HYS/HYS_3.xlsx')
HYS_SNOW_WATER_CONTENT_DF = prepare_station_df(HYS_SNOW_WATER_CONTENT_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
HYS_TEMP_AVG_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/HYS/HYS_30.xlsx')
HYS_TEMP_AVG_DF = prepare_station_df(HYS_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
HYS_TEMP_MAX_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/HYS/HYS_31.xlsx')
HYS_TEMP_MAX_DF = prepare_station_df(HYS_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
HYS_TEMP_MIN_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/HYS/HYS_32.xlsx')
HYS_TEMP_MIN_DF = prepare_station_df(HYS_TEMP_MIN_DATA)


In [121]:
# Reset value name for df merge

HYS_PRECIP_ACC_DF.rename(columns={"VALUE": "HYS_PRECIP_ACC"}, inplace=True)
HYS_PRECIP_INCR_DF.rename(columns={"VALUE": "HYS_PRECIP_INCR"}, inplace=True)
HYS_SNOW_DEPTH_DF.rename(columns={"VALUE": "HYS_SNOW_DEPTH"}, inplace=True)
HYS_SNOW_WATER_CONTENT_DF.rename(columns={"VALUE": "HYS_SNOW_WATER_CONTENT"}, inplace=True)
HYS_TEMP_AVG_DF.rename(columns={"VALUE": "HYS_TEMP_AVG"}, inplace=True)
HYS_TEMP_MAX_DF.rename(columns={"VALUE": "HYS_TEMP_MAX"}, inplace=True)
HYS_TEMP_MIN_DF.rename(columns={"VALUE": "HYS_TEMP_MIN"}, inplace=True)

In [82]:
HYS_TEMP_MIN_DF.head()

,HYS_TEMP_MIN
OBS DATE,
2008-01-01,30.0
2008-01-02,32.0
2008-01-03,28.0
2008-01-04,24.0
2008-01-05,21.0


#### Near American River, Middle Fork

In [122]:
### Duncan (DUN) ###

# PRECIPITATION, ACCUMULATED, INCHES (hourly)
DUN_PRECIP_ACC_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/DUN/DUN_2.xlsx')
DUN_PRECIP_ACC_DF = prepare_station_df(DUN_PRECIP_ACC_DATA, hourly=True)

# WIND, PEAK GUST, MPH (hourly)
DUN_WIND_GUST_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/DUN/DUN_77.xlsx')
DUN_WIND_GUST_DF = prepare_station_df(DUN_WIND_GUST_DATA, hourly=True)

# WIND, SPEED, MPH (hourly)
DUN_WIND_SPEED_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/DUN/DUN_9.xlsx')
DUN_WIND_SPEED_DF = prepare_station_df(DUN_WIND_SPEED_DATA, hourly=True)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
DUN_TEMP_AVG_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/DUN/DUN_30.xlsx')
DUN_TEMP_AVG_DF = prepare_station_df(DUN_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
DUN_TEMP_MAX_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/DUN/DUN_31.xlsx')
DUN_TEMP_MAX_DF = prepare_station_df(DUN_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
DUN_TEMP_MIN_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/DUN/DUN_32.xlsx')
DUN_TEMP_MIN_DF = prepare_station_df(DUN_TEMP_MIN_DATA)


In [123]:
# Reset value name for df merge

DUN_PRECIP_ACC_DF.rename(columns={"VALUE": "DUN_PRECIP_ACC"}, inplace=True)
DUN_WIND_GUST_DF.rename(columns={"VALUE": "DUN_WIND_GUST"}, inplace=True)
DUN_WIND_SPEED_DF.rename(columns={"VALUE": "DUN_WIND_SPEED"}, inplace=True)
DUN_TEMP_AVG_DF.rename(columns={"VALUE": "DUN_TEMP_AVG"}, inplace=True)
DUN_TEMP_MAX_DF.rename(columns={"VALUE": "DUN_TEMP_MAX"}, inplace=True)
DUN_TEMP_MIN_DF.rename(columns={"VALUE": "DUN_TEMP_MIN"}, inplace=True)

In [85]:
DUN_TEMP_MIN_DF.head()

,DUN_TEMP_MIN
OBS DATE,
2008-01-01,39.0
2008-01-02,32.0
2008-01-03,26.0
2008-01-04,NaN
2008-01-05,NaN


In [124]:
### Sugar Pine (SGP) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
SGP_PRECIP_ACC_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/SGP/SGP_2.xlsx')
SGP_PRECIP_ACC_DF = prepare_station_df(SGP_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
SGP_PRECIP_INCR_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/SGP/SGP_45.xlsx')
SGP_PRECIP_INCR_DF = prepare_station_df(SGP_PRECIP_INCR_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
SGP_TEMP_AVG_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/SGP/SGP_30.xlsx')
SGP_TEMP_AVG_DF = prepare_station_df(SGP_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
SGP_TEMP_MAX_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/SGP/SGP_31.xlsx')
SGP_TEMP_MAX_DF = prepare_station_df(SGP_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
SGP_TEMP_MIN_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/SGP/SGP_32.xlsx')
SGP_TEMP_MIN_DF = prepare_station_df(SGP_TEMP_MIN_DATA)


In [125]:
# Reset value name for df merge

SGP_PRECIP_ACC_DF.rename(columns={"VALUE": "SGP_PRECIP_ACC"}, inplace=True)
SGP_PRECIP_INCR_DF.rename(columns={"VALUE": "SGP_PRECIP_INCR"}, inplace=True)
SGP_TEMP_AVG_DF.rename(columns={"VALUE": "SGP_TEMP_AVG"}, inplace=True)
SGP_TEMP_MAX_DF.rename(columns={"VALUE": "SGP_TEMP_MAX"}, inplace=True)
SGP_TEMP_MIN_DF.rename(columns={"VALUE": "SGP_TEMP_MIN"}, inplace=True)

In [88]:
SGP_TEMP_MIN_DF.head()

,SGP_TEMP_MIN
OBS DATE,
2008-01-01,27.0
2008-01-02,27.0
2008-01-03,34.0
2008-01-04,32.0
2008-01-05,31.0


#### Near American River, South Fork

In [126]:
### Forni Ridge (FRN) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
FRN_PRECIP_ACC_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/FRN/FRN_2.xlsx')
FRN_PRECIP_ACC_DF = prepare_station_df(FRN_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
FRN_PRECIP_INCR_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/FRN/FRN_45.xlsx')
FRN_PRECIP_INCR_DF = prepare_station_df(FRN_PRECIP_INCR_DATA)

# SNOW DEPTH, INCHES (daily)
FRN_SNOW_DEPTH_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/FRN/FRN_18.xlsx')
FRN_SNOW_DEPTH_DF = prepare_station_df(FRN_SNOW_DEPTH_DATA)

# SNOW, WATER CONTENT, INCHES (daily)
FRN_SNOW_WATER_CONTENT_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/FRN/FRN_3.xlsx')
FRN_SNOW_WATER_CONTENT_DF = prepare_station_df(FRN_SNOW_WATER_CONTENT_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
FRN_TEMP_AVG_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/FRN/FRN_30.xlsx')
FRN_TEMP_AVG_DF = prepare_station_df(FRN_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
FRN_TEMP_MAX_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/FRN/FRN_31.xlsx')
FRN_TEMP_MAX_DF = prepare_station_df(FRN_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
FRN_TEMP_MIN_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/FRN/FRN_32.xlsx')
FRN_TEMP_MIN_DF = prepare_station_df(FRN_TEMP_MIN_DATA)


In [127]:
# Reset value name for df merge

FRN_PRECIP_ACC_DF.rename(columns={"VALUE": "FRN_PRECIP_ACC"}, inplace=True)
FRN_PRECIP_INCR_DF.rename(columns={"VALUE": "FRN_PRECIP_INCR"}, inplace=True)
FRN_SNOW_DEPTH_DF.rename(columns={"VALUE": "FRN_SNOW_DEPTH"}, inplace=True)
FRN_SNOW_WATER_CONTENT_DF.rename(columns={"VALUE": "FRN_SNOW_WATER_CONTENT"}, inplace=True)
FRN_TEMP_AVG_DF.rename(columns={"VALUE": "FRN_TEMP_AVG"}, inplace=True)
FRN_TEMP_MAX_DF.rename(columns={"VALUE": "FRN_TEMP_MAX"}, inplace=True)
FRN_TEMP_MIN_DF.rename(columns={"VALUE": "FRN_TEMP_MIN"}, inplace=True)

In [91]:
FRN_TEMP_MIN_DF.head()

,FRN_TEMP_MIN
OBS DATE,
2008-01-01,37.0
2008-01-02,32.0
2008-01-03,25.0
2008-01-04,28.0
2008-01-05,NaN


In [128]:
### Pacific House (PFH) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
PFH_PRECIP_ACC_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/PFH/PFH_2.xlsx')
PFH_PRECIP_ACC_DF = prepare_station_df(PFH_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
PFH_PRECIP_INCR_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/PFH/PFH_45.xlsx')
PFH_PRECIP_INCR_DF = prepare_station_df(PFH_PRECIP_INCR_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
PFH_TEMP_AVG_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/PFH/PFH_30.xlsx')
PFH_TEMP_AVG_DF = prepare_station_df(PFH_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
PFH_TEMP_MAX_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/PFH/PFH_31.xlsx')
PFH_TEMP_MAX_DF = prepare_station_df(PFH_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
PFH_TEMP_MIN_DATA = pd.read_excel(f'{PROJECT_PATH}/CDEC_Water_Data/PFH/PFH_32.xlsx')
PFH_TEMP_MIN_DF = prepare_station_df(PFH_TEMP_MIN_DATA)


In [129]:
# Reset value name for df merge

PFH_PRECIP_ACC_DF.rename(columns={"VALUE": "PFH_PRECIP_ACC"}, inplace=True)
PFH_PRECIP_INCR_DF.rename(columns={"VALUE": "PFH_PRECIP_INCR"}, inplace=True)
PFH_TEMP_AVG_DF.rename(columns={"VALUE": "PFH_TEMP_AVG"}, inplace=True)
PFH_TEMP_MAX_DF.rename(columns={"VALUE": "PFH_TEMP_MAX"}, inplace=True)
PFH_TEMP_MIN_DF.rename(columns={"VALUE": "PFH_TEMP_MIN"}, inplace=True)

In [94]:
PFH_TEMP_MIN_DF.head()

,PFH_TEMP_MIN
OBS DATE,
2008-01-01,30.0
2008-01-02,29.0
2008-01-03,29.0
2008-01-04,32.0
2008-01-05,32.0


### Cleaning CDEC Water data 

To do (see steps in Notion task as well): 
- Smoothing: 
    - Exponential smoothing for this problem (I think): treat more recent data as more informative data, in which case exponential smoothing is a good option. 
- Handle seasonality --> seasonal data smoothing. See Orielly book. Can be used with exponential moving avg smoothing
- Normalize data

In [95]:
STATION_DF = [ADR_PRECIP_ACC_DF, ADR_PRECIP_INCR_DF, ADR_TEMP_AVG_DF, ADR_TEMP_MAX_DF, ADR_TEMP_MIN_DF, HYS_PRECIP_ACC_DF, HYS_PRECIP_INCR_DF, HYS_SNOW_DEPTH_DF, HYS_SNOW_WATER_CONTENT_DF, HYS_TEMP_AVG_DF, HYS_TEMP_MAX_DF, HYS_TEMP_MIN_DF, DUN_PRECIP_ACC_DF, DUN_WIND_GUST_DF, DUN_WIND_SPEED_DF, DUN_TEMP_AVG_DF, DUN_TEMP_MAX_DF, DUN_TEMP_MIN_DF, SGP_PRECIP_ACC_DF, SGP_PRECIP_INCR_DF, SGP_TEMP_AVG_DF, SGP_TEMP_MAX_DF, SGP_TEMP_MIN_DF, FRN_PRECIP_ACC_DF, FRN_PRECIP_INCR_DF, FRN_SNOW_DEPTH_DF, FRN_SNOW_WATER_CONTENT_DF, FRN_TEMP_AVG_DF, FRN_TEMP_MAX_DF, FRN_TEMP_MIN_DF, PFH_PRECIP_ACC_DF, PFH_PRECIP_INCR_DF, PFH_TEMP_AVG_DF, PFH_TEMP_MAX_DF, PFH_TEMP_MIN_DF]

In [135]:
def handle_missing(df):
  # forward fill missing values
  df.ffill(axis=0, inplace=True)
  print("Null/NaN count: ", df.isnull().sum().sum())

In [136]:
"""
Smoothing utility function: Exponential moving average

- Uses past 30 periods or days to inform prediction (give them more weight) in order to capture trends
- Span corresponds to what is commonly called an “N-day EW moving average”
- mean() for the average in exponential moving average
"""

def smooth(df):
  # cast value column to int for smoothing
  # df.iloc[:, 0] = df.iloc[:, 0].astype(int)
  return df.ewm(span=30.0, ignore_na=True).mean(engine='numba')

In [137]:
for df in STATION_DF: 
    handle_missing(df)
    # smooth(df)

Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0
Null/NaN count:  0


In [138]:
# Use and remove seasonality: Use differencing. See page 121 in the ML book phone. With code example. 
# Seasonal data smoothing see Orielly book 
# ADD HERE: 


In [140]:
# Normalizing utility function - prep for training. Last step

# see your Notion notes
# maximum absolute scaling rescales each feature between -1 and 1 by dividing every observation 
# by its maximum absolute value


# ADD HERE: 

